In [9]:
#Importing libraries

In [11]:
import numpy as np
import os
import pickle as pkl
from tqdm import tqdm
import tensorflow as tf
from tensorflow.keras.applications.resnet50 import ResNet50, preprocess_input
from tensorflow.keras.preprocessing import image
from tensorflow.keras.layers import GlobalMaxPool2D, Dense, Input, Lambda
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
import faiss 
from sklearn.model_selection import train_test_split

In [12]:
#Extract filenames from folder

In [15]:
image_dir = 'data/images'
filenames = [os.path.join(image_dir, f) for f in os.listdir(image_dir) if f.endswith(('.jpg', '.png'))]
print(f"Total images found: {len(filenames)}")

Total images found: 44441


In [17]:
#Define feature extractor model with fine-tuning capability

In [19]:
def build_feature_extractor(finetune=False):
    base_model = ResNet50(weights='imagenet', include_top=False, input_shape=(224,224,3))
    
    if not finetune:
        base_model.trainable = False
    else:
        for layer in base_model.layers[:-20]:
            layer.trainable = False
        for layer in base_model.layers[-20:]:
            layer.trainable = True
    
    x = base_model.output
    x = GlobalMaxPool2D()(x)  
    x = tf.keras.layers.Lambda(lambda t: tf.math.l2_normalize(t, axis=1))(x)  # L2 normalize embeddings
    
    model = Model(inputs=base_model.input, outputs=x)
    return model

feature_extractor = build_feature_extractor(finetune=False)
feature_extractor.summary()


Model: "functional"

┏━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━┳━━━━━━━━━━━━━━━━━━━━━━━━━━━━┓
┃ Layer (type)                  ┃ Output Shape              ┃         Param # ┃ Connected to               ┃
┡━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━╇━━━━━━━━━━━━━━━━━━━━━━━━━━━━┩
│ input_layer (InputLayer)      │ (None, 224, 224, 3)       │               0 │ -                          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_pad (ZeroPadding2D)     │ (None, 230, 230, 3)       │               0 │ input_layer[0][0]          │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_conv (Conv2D)           │ (None, 112, 112, 64)      │           9,472 │ conv1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_bn (BatchNormalization) │ (None, 112, 112, 64)      │             256 │ conv1_conv[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv1_relu (Activation)       │ (None, 112, 112, 64)      │               0 │ conv1_bn[0][0]             │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pad (ZeroPadding2D)     │ (None, 114, 114, 64)      │               0 │ conv1_relu[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ pool1_pool (MaxPooling2D)     │ (None, 56, 56, 64)        │               0 │ pool1_pad[0][0]            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_conv (Conv2D)  │ (None, 56, 56, 64)        │           4,160 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_1_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_1_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_1_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_conv (Conv2D)  │ (None, 56, 56, 64)        │          36,928 │ conv2_block1_1_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_bn             │ (None, 56, 56, 64)        │             256 │ conv2_block1_2_conv[0][0]  │
│ (BatchNormalization)          │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_2_relu           │ (None, 56, 56, 64)        │               0 │ conv2_block1_2_bn[0][0]    │
│ (Activation)                  │                           │                 │                            │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_0_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ pool1_pool[0][0]           │
├───────────────────────────────┼───────────────────────────┼─────────────────┼────────────────────────────┤
│ conv2_block1_3_conv (Conv2D)  │ (None, 56, 56, 256)       │          16,640 │ conv2_block1_2_relu[0][0]  │
├───────────────────────────────┼───────────────────────────┼───────────────

 Total params: 23,587,712 (89.98 MB)

 Trainable params: 0 (0.00 B)

 Non-trainable params: 23,587,712 (89.98 MB)

In [21]:
#Extract features function

In [23]:
def extract_features(image_paths, model, batch_size=32):
    features = []
    for i in tqdm(range(0, len(image_paths), batch_size)):
        batch_paths = image_paths[i:i+batch_size]
        batch_images = []
        for path in batch_paths:
            img = image.load_img(path, target_size=(224,224))
            img_array = image.img_to_array(img)
            batch_images.append(img_array)
        batch_images = preprocess_input(np.array(batch_images))
        batch_features = model.predict(batch_images)
        features.append(batch_features)
    features = np.vstack(features)
    return features

In [ ]:
features = extract_features(filenames, feature_extractor)
np.save('features/image_features.npy', features)
with open('features/filenames.pkl', 'wb') as f:
    pkl.dump(filenames, f)

  0%|                                                                                         | 0/1389 [00:00<?, ?it/s]

1/1 ━━━━━━━━━━━━━━━━━━━━ 9s 9s/step


  0%|                                                                               | 1/1389 [00:11<4:34:39, 11.87s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 2s 2s/step


  0%|                                                                               | 2/1389 [00:14<2:33:49,  6.65s/it]

...

 10%|███████▊                                                                     | 140/1389 [06:54<1:43:17,  4.96s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


 10%|███████▊                                                                     | 141/1389 [06:58<1:34:20,  4.54s/it]

1/1 ━━━━━━━━━━━━━━━━━━━━ 3s 3s/step


 10%|███████▊                                                                     | 142/1389 [07:02<1:32:13,  4.44s/it]

In [ ]:
features = np.load('features/image_features.npy')
with open('features/filenames.pkl', 'rb') as f:
    filenames = pkl.load(f)

print(f"Feature shape: {features.shape}")

In [ ]:
#Build FAISS index for fast nearest neighbor search

In [ ]:
embedding_dim = features.shape[1]

In [ ]:
index = faiss.IndexFlatL2(embedding_dim)
index.add(features)  # Add all feature vectors to index

print(f"Total indexed vectors: {index.ntotal}")

In [ ]:
def triplet_loss(y_true, y_pred, margin=0.3):
    # y_pred shape: (batch_size, 3*embedding_dim) concatenated anchor, positive, negative
    anchor, positive, negative = y_pred[:, :embedding_dim], y_pred[:, embedding_dim:2*embedding_dim], y_pred[:, 2*embedding_dim:]
    
    pos_dist = tf.reduce_sum(tf.square(anchor - positive), axis=1)
    neg_dist = tf.reduce_sum(tf.square(anchor - negative), axis=1)
    
    loss = tf.maximum(pos_dist - neg_dist + margin, 0.0)
    return tf.reduce_mean(loss)

# Example input: 3 images per sample (anchor, positive, negative)
def build_triplet_model(feature_extractor):
    input_anchor = Input((224,224,3))
    input_positive = Input((224,224,3))
    input_negative = Input((224,224,3))
    
    feat_anchor = feature_extractor(input_anchor)
    feat_positive = feature_extractor(input_positive)
    feat_negative = feature_extractor(input_negative)
    
    merged = tf.keras.layers.Concatenate(axis=1)([feat_anchor, feat_positive, feat_negative])
    model = Model(inputs=[input_anchor, input_positive, input_negative], outputs=merged)
    
    model.compile(optimizer=Adam(1e-4), loss=triplet_loss)
    return model

In [ ]:
#Search function example

In [ ]:
img = image.load_img(image_path, target_size=(224,224))
    img_array = image.img_to_array(img)
    img_array = preprocess_input(np.expand_dims(img_array, axis=0))
    
    query_feature = model.predict(img_array)
    query_feature = query_feature / np.linalg.norm(query_feature)
    
    distances, indices = faiss_index.search(query_feature, top_k)
    return distances[0], [filenames[i] for i in indices[0]]

In [ ]:
#Example

In [ ]:
query_img = filenames[0]  # Just for demo
distances, recommendations = recommend(query_img, feature_extractor, index, filenames)

print("Recommended images:")
for i, img_path in enumerate(recommendations[1:], start=1):
    print(f"{i}. {img_path} (Distance: {distances[i]:.4f})")